In [1]:
from PyPDF2 import PdfReader

reader = PdfReader("series.pdf")
text = ""
for page in reader.pages:
    text += page.extract_text()

with open("series.txt", "w") as f:
    f.write(text)

In [2]:
# series to data
import re

data = []

with open('series.txt', 'r') as f:
    print('open')

    for line in f:
        
        # Look for the gender
        if re.search(r'Fem.*', line):
            gender = 'Femenino'
        if re.search(r'Masc.*', line):
            gender = 'Masculino'
        
        # Look for the prueba
        if (p := re.search(r'Prueba\s.+\.,\s*(\d{2,4}m\s)\s*(Libre|Espalda|Braza|Mariposa|Estilos).*', line)):
            prueba = p.group(1) + p.group(2)
        
        # Look for the name of the swimmer
        if (s := re.search(r'(10|[0-9])((?:[A-ZÑÜ\-]+\s*)+,\s[A-ZÑ][a-zñ]+(?:[- ][A-ZÑ][a-zñ]+)*)\s\d{2}\s(.*)\s(\d{1,2}:\d{2}\.\d{2}|\d{2}\.\d{2}|NT).*', line)):
            swimmer = s.group(2)
            club = s.group(3)
            time = s.group(4)
            data.append((swimmer, gender, club, prueba, time))
        print('it')
data

: 

In [ ]:
import pandas as pd

df = pd.DataFrame(data, columns= ['Name', 'Gender','Club', 'Prueba', 'Time'])
df

,Name,Gender,Club,Prueba,Time
0,"GARCIA HERNANDEZ, Ines",Femenino,C. Alameda De Osuna,50m Libre,30.02
1,"POP MARTIN, Alexia",Femenino,C.M.N. Arganda,50m Libre,28.96
2,"BLANCO PEÑAS, Vega",Femenino,C.N. Jimenez,50m Libre,29.00
3,"VERDEJO RODRIGUEZ, Paula Haotong",Femenino,C.N. Vallmont,50m Libre,NT
4,"CONTRERAS CASTILLO, Sandra",Femenino,C.N. San Blas,50m Libre,28.60
...,...,...,...,...,...
442,"ALVAREZ PARRA, Lucia",Femenino,A.D.N. Mostoles,400m Libre,4:42.73
443,"TOMAS PRIETO, Rocio",Femenino,A.D.N. Mostoles,400m Libre,4:37.65
444,"VILLANUEVA MARTINEZ, Maria",Femenino,C.M.N. Arganda,400m Libre,4:37.72
445,"BENITO BARCA, Lucia",Femenino,C.N. Ciudad De Alcorcon,400m Libre,4:51.77


In [ ]:
df[df['Club']=='Safa Madrid']

,Name,Gender,Club,Prueba,Time
25,"FERNANDEZ BLAZQUEZ, Victor",Masculino,Safa Madrid,50m Libre,23.74
26,"FERNANDEZ BLAZQUEZ, Pablo",Masculino,Safa Madrid,50m Libre,23.15
46,"ALDERETE GARCIA, Rodrigo",Masculino,Safa Madrid,1500m Libre,1046.15
71,"GUTIERREZ MADERUELO, Javier",Masculino,Safa Madrid,50m Braza,29.40
94,"GARCIA NAVAS, Pablo",Masculino,Safa Madrid,100m Mariposa,59.34
103,"FERNANDEZ BLAZQUEZ, Pablo",Masculino,Safa Madrid,100m Mariposa,56.20
127,"FERNANDEZ BLAZQUEZ, Victor",Masculino,Safa Madrid,100m Espalda,59.09
130,"RODRIGUEZ CRESPO, Alejandro",Masculino,Safa Madrid,100m Espalda,60.53
159,"GARCIA GONZALEZ, Carlos",Masculino,Safa Madrid,200m Estilos,131.10
182,"OCAÑA MARTIN, Pedro",Masculino,Safa Madrid,200m Libre,124.84


In [ ]:
df['Time'].replace('NT', '59:59.59', inplace= True)

C:\Users\jesus\AppData\Local\Temp\ipykernel_24456\2302876258.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Time'].replace('NT', '59:59.59', inplace= True)


In [ ]:
df['Time'] = df['Time'].apply(lambda x: int(x[0:x.index(':')]) * 60 + int(x[x.index(':')+1:x.index('.')]) + float('0.'+x[-2:]) if len(x) > 5 else float(x))

In [ ]:
gender = 'Masculino'

df = df[df['Gender'] == gender]

points = {club : 0 for club in df['Club'].unique()}

for prueba in df['Prueba'].unique():

    swimmers = df[(df['Prueba'] == prueba) & (df['Gender'] == gender)]

    swimmers.sort_values(by= 'Time', ascending= True, inplace= True)

    for i in range(swimmers.shape[0]):

        linea = swimmers.iloc[i]
        points[linea['Club']] += max(0, 16-i)


points

C:\Users\jesus\AppData\Local\Temp\ipykernel_24456\4268368797.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  swimmers.sort_values(by= 'Time', ascending= True, inplace= True)


{'C.N. Mirasierra': 288,
 'A.D. Rivas Natación': 244,
 'C.N. San Fernando De Henares': 179,
 'C.N. Coslada': 219,
 'C.N. Colegio Mirasur': 288,
 'C.N. San Blas': 253,
 'Safa Madrid': 244,
 'C.D. Sek': 366}

In [ ]:
results = sorted(points.items(), key= lambda x: -x[1])
results

[('C.N. Colegio Mirasur', 300),
 ('C.D. Sek', 291),
 ('C.N. Mirasierra', 289),
 ('C.N. San Blas', 267),
 ('A.D. Rivas Natación', 260),
 ('Safa Madrid', 255),
 ('C.N. Coslada', 199),
 ('C.N. San Fernando De Henares', 168)]

[30, 26, 30, 30, 25, 24, 21, 22]